In [1]:
cd 'deep-learning-models/'

/data/fs4/home/dgrossman/deep-learning-models


In [2]:
import numpy as np
from resnet50 import ResNet50
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions
from keras.models import Model
import scipy.spatial.distance
import time
import json


Using TensorFlow backend.


In [3]:
def load_image(img_path):
    data = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(data)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [4]:
#def get_base_model():
#    base_model = ResNet50(weights='imagenet',include_top=False)
#    return base_model

def get_models():
    # include_top needs to be True for this to work
    base_model = ResNet50(weights='imagenet',include_top=True)
    model = Model(input=base_model.input, output=base_model.get_layer('flatten_1').output)
    return (model, base_model)

def image_features(img,model):
    features = np.zeros((1,2048),dtype=np.float16)
    #model = Model(input=base_model.input, output=base_model.get_layer('flatten_1').output)
    predictions = model.predict(img)
    return predictions

In [5]:
model, base_model = get_models()

K.image_dim_ordering: tf


In [ ]:
# model = Model(input=bm.input, output=bm.get_layer('flatten_1').output)

In [ ]:
# print(scipy.spatial.distance.cosine(feature1,feature2),'good')
# print(scipy.spatial.distance.cosine(feature1,feature3),'bad')
# print(scipy.spatial.distance.cosine(feature2,feature3),'bad')

In [6]:
prefix = '/local_data/dgrossman/VeRi/'


In [ ]:
tLines = open(prefix + 'trainingLines','r')
trainingList = list()
for tLine in tLines:
    tLine = tLine.strip()
    tLine = tLine.replace('"','')
    parts = tLine.split(' ')
    ldict = dict()
    for part in parts:
        l, r = part.split('=')
        ldict[l]=str(r)
    trainingList.append(ldict)
tLines.close()

In [7]:
trainingList = list()
tLines = open(prefix + 'test_features.json','r')
for line in tLines:
    line = line.strip()
    line = json.loads(line)
    trainingList.append(line)

In [8]:
trainingList[0]

{'cameraID': 'c002',
 'colorID': '-1',
 'imageName': '0002_c002_00030600_0.jpg',
 'typeID': '-1',
 'vehicleID': '0002'}

In [10]:
import json
#outFile = open('/local_data/dgrossman/VeRi/training_features.json','w')
outFile = open('/local_data/dgrossman/VeRi/test_features.json','w')

batchSize = 1000
start = time.time()
for idx,line in enumerate(trainingList):
    tempd = dict()
    if idx % batchSize == 0:
        end = time.time() - start
        start = time.time()
        print ('total {0} batch {1} images in {2} seconds'.format(idx,batchSize,end))
    #img = load_image(prefix + 'image_train/'+line['imageName'])
    img = load_image(prefix + 'image_test/'+line['imageName'])
    feature = image_features(img, model) 
    tempd['resnet50'] = feature.tolist()[0]
    tempd.update(line)
    outFile.write(json.dumps(tempd)+'\n')
outFile.close()

total 0 batch 1000 images in 0.00019979476928710938 seconds
total 1000 batch 1000 images in 19.921246767044067 seconds
total 2000 batch 1000 images in 19.465110778808594 seconds
total 3000 batch 1000 images in 18.9404559135437 seconds
total 4000 batch 1000 images in 19.13255214691162 seconds
total 5000 batch 1000 images in 18.967551469802856 seconds
total 6000 batch 1000 images in 19.22211003303528 seconds
total 7000 batch 1000 images in 19.369484424591064 seconds
total 8000 batch 1000 images in 18.831890106201172 seconds
total 9000 batch 1000 images in 18.86261248588562 seconds
total 10000 batch 1000 images in 19.05212092399597 seconds
total 11000 batch 1000 images in 18.88049077987671 seconds


In [ ]:
funtime = open('/local_data/dgrossman/VeRi/training_features.json','r')
work = list()
for line in funtime:
    line = line.strip()
    line = json.loads(line)
    work.append(line)
funtime.close()

In [ ]:
work[0]
